In [1]:
import pandas as pd
import numpy as np
import datetime
import numpy as np
from haversine import haversine
import json
import os
print(datetime.datetime.now())
print(os.getcwd())
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

2020-04-06 13:17:19.466973
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Competitor_conquest


In [2]:
df_all_competitors=pd.read_csv("./BL_competitor_list_with_geo_location_JL_2020-04-03.csv",dtype=str)
df_all_competitors['Google_lat']=df_all_competitors['Google_lat'].astype(float)
df_all_competitors['Google_lng']=df_all_competitors['Google_lng'].astype(float)

df_all_competitors['zip_code']=df_all_competitors['zip_code'].apply(lambda x: str(x).zfill(5))
df_all_competitors['Google_zip']=df_all_competitors['Google_zip'].apply(lambda x: str(x).zfill(5))

In [3]:
df_all_competitors.head(2)

,store_name,address,city,state,zip_code,status,search_key,Google_Address,Google_lat,Google_lng,Google_zip
0,Bed Bath & Beyond,700 meyerland plaza,houston,tx,77096,Closed,"700 meyerland plaza, houston, tx, 77096","700 Meyerland Plaza, Houston, TX 77096, USA",29.687731,-95.461461,77096
1,Bed Bath & Beyond,2306 highway 6 south,houston,tx,77077,Closed,"2306 highway 6 south, houston, tx, 77077","2306 S Texas 6, Houston, TX 77077, USA",29.738768,-95.644768,77077


In [4]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",
                            dtype=str,sep="|")
df_store_list=df_store_list[['location_id','latitude_meas','longitude_meas']]
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)

In [6]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
#
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)
    
#   
df_output=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
df_output_2=df_output[pd.isnull(df_output['zip_cd'])]
df_output_1=df_output[pd.notnull(df_output['zip_cd'])]
print(df_output_2.shape)

del df_output_2['zip_cd']
del df_output_2['zip_type']
#

df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in df_output_2.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"
print(df_10_zips_all_new_stores['location_id'].nunique())
#
df_new_store_zips=df_10_zips_all_new_stores[['location_id','zip_cd','zip_type']]
df_output_2=pd.merge(df_output_2,df_new_store_zips,how="left")
df_output=df_output_1.append(df_output_2)

(34, 5)
34


In [7]:
df_output['zip_type']=df_output['zip_type'].replace("zip_10","P")
df_output_P=df_output[df_output['zip_type']=="P"]
df_output_P=df_output_P[['location_id','zip_cd','zip_type']].drop_duplicates()

In [8]:
df_BL_store_info=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",
                               dtype=str,sep="|")
df_SOTF_list=pd.ExcelFile("./Store List Report 03.06.20 409pm.xlsx")
df_SOTF_list=df_SOTF_list.parse("Store List",dtype=str)
df_SOTF_list=df_SOTF_list[['Store','SOTF']]
df_SOTF_list=df_SOTF_list[pd.notnull(df_SOTF_list['Store'])]
df_SOTF_list=df_SOTF_list.rename(columns={"Store":"location_id"})
df_SOTF_list=df_SOTF_list.drop_duplicates("location_id")

In [9]:
df_SOTF_list.shape

(1485, 2)

In [10]:
df_all_competitors.groupby("store_name")['search_key'].nunique().to_frame().reset_index()

,store_name,search_key
0,Art Van Furniture,142
1,At Home,218
2,Bed Bath & Beyond,989
3,Macy's,617
4,Pier 1,1166


In [11]:
df_all_competitors.head(2)

,store_name,address,city,state,zip_code,status,search_key,Google_Address,Google_lat,Google_lng,Google_zip
0,Bed Bath & Beyond,700 meyerland plaza,houston,tx,77096,Closed,"700 meyerland plaza, houston, tx, 77096","700 Meyerland Plaza, Houston, TX 77096, USA",29.687731,-95.461461,77096
1,Bed Bath & Beyond,2306 highway 6 south,houston,tx,77077,Closed,"2306 highway 6 south, houston, tx, 77077","2306 S Texas 6, Houston, TX 77077, USA",29.738768,-95.644768,77077


In [12]:
df_competitor_zips_20=pd.DataFrame()
for ind,row in df_all_competitors.iterrows():
    store_center=(row['Google_lat'],row['Google_lng'])
    store_add=row['search_key']
    store_name=row['store_name']
    store_zip_list_20=[]
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi")
        if dist<=20:
            store_zip_list_20.append(zip_cd)
    df=pd.DataFrame({"search_key":[store_add]*len(store_zip_list_20),
                     "store_name":[store_name]*len(store_zip_list_20),
                     "zip_cd":store_zip_list_20})
    df_competitor_zips_20=df_competitor_zips_20.append(df)
df_competitor_zips_20=df_competitor_zips_20.drop_duplicates()
print("df_competitor_zips_20.shape",df_competitor_zips_20.shape)



df_competitor_zips_20.shape (319385, 3)


In [13]:
df_1_comp_stores_paired_P=pd.merge(df_all_competitors,df_output_P,left_on="zip_code",right_on="zip_cd",how="left")
df_1_comp_stores_paired_P=df_1_comp_stores_paired_P[df_1_comp_stores_paired_P['zip_type']=="P"]
del df_1_comp_stores_paired_P['zip_cd']
df_1_comp_stores_paired_P=df_1_comp_stores_paired_P.rename(columns={"zip_type":"paired_condition"})
print("df_1_comp_stores_paired_P.shape","df_1_comp_stores_paired_P['search_key'].nunique()")
print(df_1_comp_stores_paired_P.shape,df_1_comp_stores_paired_P['search_key'].nunique())

df_paired_20=pd.DataFrame()
for ind,row in df_all_competitors.iterrows():
    store_center=(row['Google_lat'],row['Google_lng'])
    search_key=row['search_key']
    store_name=row['store_name']
    
    BL_store_list_20=[]
    for ind_BL,row_BL in df_store_list.iterrows():
        BL_store_location=row_BL['location_id']
        BL_store_center=(row_BL['latitude_meas'],row_BL['longitude_meas'])
        dist=haversine(store_center,BL_store_center,unit="mi")
        if dist<=20:
            BL_store_list_20.append(BL_store_location)
    df=pd.DataFrame({"search_key":[search_key]*len(BL_store_list_20),
                     "store_name":[store_name]*len(BL_store_list_20),
                     "location_id":BL_store_list_20})
    df_paired_20=df_paired_20.append(df)
print("df_paired_20.shape","df_paired_20['search_key'].nunique()")
print(df_paired_20.shape,df_paired_20['search_key'].nunique())

df_2_comp_stores_paired_20=pd.merge(df_all_competitors,df_paired_20,on=['store_name',"search_key"],how="left")
df_2_comp_stores_paired_20['paired_condition']="dist_20"


df_1_comp_stores_paired_P.shape df_1_comp_stores_paired_P['search_key'].nunique()
(2166, 13) 1833
df_paired_20.shape df_paired_20['search_key'].nunique()
(16876, 3) 2943


In [14]:
df_comp_stores_paired=df_1_comp_stores_paired_P.append(df_2_comp_stores_paired_20)
    
df_store_list_rename=df_store_list.rename(columns={"latitude_meas":"BL_latitude_meas","longitude_meas":"BL_longitude_meas"})
df_comp_stores_paired=pd.merge(df_comp_stores_paired,df_store_list_rename,on="location_id",how="left")

df_comp_stores_paired['store_dist']=df_comp_stores_paired.apply(lambda x: haversine([x['Google_lat'],x['Google_lng']],[x['BL_latitude_meas'],x['BL_longitude_meas']],unit='mi'),axis=1)

df_competitor_zips_20=df_competitor_zips_20.rename(columns={"zip_cd":"comp_target_zip"})
df_output_detail=pd.merge(df_comp_stores_paired,df_competitor_zips_20,on=['store_name',"search_key"],how="left")

df_output_detail=df_output_detail.reset_index()
del df_output_detail['index']
df_output_detail['dist_comp_zip_to_BL_store']=df_output_detail.apply(lambda x: haversine((x['BL_latitude_meas'],x['BL_longitude_meas']),zip_centers[x['comp_target_zip']],unit="mi"),axis=1)
print("df_output_detail.shape","df_output_detail['search_key'].nunique()")
print(df_output_detail.shape,df_output_detail['search_key'].nunique())
df_output=df_output_detail[df_output_detail['dist_comp_zip_to_BL_store']<=30]

df_zips_with_stores=df_output[['store_name','search_key','Google_Address','location_id','comp_target_zip']].drop_duplicates()
df_unique_zips=df_output[['comp_target_zip']].drop_duplicates()



df_output_detail.shape df_output_detail['search_key'].nunique()
(2580213, 18) 3123


In [21]:
df_BL_associated=df_output[['location_id','store_name','status']].drop_duplicates()
df_BL_associated=pd.merge(df_BL_associated,df_BL_store_info,on="location_id",how="left")
df_BL_associated=pd.merge(df_BL_associated,df_SOTF_list,on="location_id",how="left")

In [43]:
df_open_SOTF=pd.merge(df_store_list,df_SOTF_list,on="location_id",how="left")
print(df_open_SOTF.shape,df_open_SOTF['location_id'].nunique())
df_open_SOTF=df_open_SOTF[['location_id','SOTF']].drop_duplicates()
# sotf 2020 labeled as no, for now
df_open_SOTF['SOTF']=df_open_SOTF['SOTF'].replace("Yes 2020","Yes")
print(df_open_SOTF['SOTF'].unique())
df_open_SOTF.shape

(1404, 4) 1404
['No' 'Yes']


(1404, 2)

In [51]:
dict_order_index={"open":1,"Closed":2,"impendingclose":3,"balance":4}

In [70]:
writer=pd.ExcelWriter("./BL_competitor_store_matrix_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
for brand, df_group in df_output.groupby("store_name"):
    df_brand_store=df_group[['status','location_id']].drop_duplicates()
    df_brand_store=pd.merge(df_brand_store,df_open_SOTF,on="location_id",how="outer")
    df_brand_store['status']=df_brand_store['status'].fillna("balance")
    df_brand_store=df_brand_store.replace("Yes","SOTF").replace("No","Legacy")
    df_brand_store_list=df_brand_store[['status','location_id','SOTF']].drop_duplicates()
    df_brand_store_list.insert(0,'store_name',brand)
    
    df_brand_store=df_brand_store.groupby(["status","SOTF"])['location_id'].nunique().to_frame().reset_index()    
    df_brand_store=df_brand_store.pivot_table(index="status",columns="SOTF",values="location_id").reset_index()
    df_brand_store['index']=df_brand_store['status'].apply(lambda x: dict_order_index[x])
    df_brand_store=df_brand_store.sort_values("index")
    df_brand_store.insert(0,'store_name',brand)
    del df_brand_store['index']
    
    df_brand_store.to_excel(writer,brand+"_BL_matrix",index=False)
    df_brand_store_list.to_excel(writer,brand+"_BL_list",index=False)
    
writer.save()

In [69]:
import os
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Competitor_conquest'